**PROJET DATAMINING**  -  *YANIS PIRES PORTELADA - SANJAY CANDA*

**ETAPE 1 :** Récupération de données à partir de **Wikidata**
Dans cette première étape, nous allons récupérer des données à partir de Wikidata en utilisant **SPARQL.**
Nous allons interroger **l'endpoint** de Wikidata pour obtenir une liste de *grandes villes, leurs pays et une image associée*.

In [2]:
import numpy as np
import pandas as pd
import matplotlib as plot
import SPARQLWrapper as sw
import os

In [ ]:
import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

nombre_villes = 10

# Get cities
def get_cities(nombre_villes):
    query = """SELECT DISTINCT ?grandeville ?grandevilleLabel ?pays ?paysLabel ?image {
    ?grandeville wdt:P31 wd:Q1549591;
                wdt:P17 ?pays;
                wdt:P18 ?image.
    SERVICE wikibase:label { bd:serviceParam wikibase:language "fr". }
    }
    LIMIT """ + str(nombre_villes)
    return query

query = get_cities(nombre_villes)

def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (
        sys.version_info[0],
        sys.version_info[1],
    )
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


array = []
results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    array.append(
        (
            result["grandevilleLabel"]["value"],
            result["paysLabel"]["value"],
            result["image"]["value"],
        )
    )


"""-------------------------------------------------------------------------
Affiche le dataframe contenant les villes, les pays et les images associées.
----------------------------------------------------------------------------"""


dataframe = pd.DataFrame(array, columns=["ville", "pays", "image"])
dataframe = dataframe.astype(
    dtype={"ville": "<U200", "pays": "<U200", "image": "<U200"}
)


#affichage du dataframe
dataframe

,ville,pays,image
0,Krefeld,Allemagne,http://commons.wikimedia.org/wiki/Special:File...
1,Francfort-sur-le-Main,Allemagne,http://commons.wikimedia.org/wiki/Special:File...
2,Malmö,Suède,http://commons.wikimedia.org/wiki/Special:File...
3,Helsinki,Finlande,http://commons.wikimedia.org/wiki/Special:File...
4,Innsbruck,Autriche,http://commons.wikimedia.org/wiki/Special:File...
5,Wuppertal,Allemagne,http://commons.wikimedia.org/wiki/Special:File...
6,Parme,Italie,http://commons.wikimedia.org/wiki/Special:File...
7,Leipzig,Allemagne,http://commons.wikimedia.org/wiki/Special:File...
8,Sarrebruck,Allemagne,http://commons.wikimedia.org/wiki/Special:File...
9,Bremerhaven,Allemagne,http://commons.wikimedia.org/wiki/Special:File...


**ETAPE 2 :** Téléchargement des données

In [ ]:
import requests
import shutil
import time

def download_image(url):
    
    time.sleep(1) # Pause de 1 seconde entre les téléchargements pour éviter de surcharger le serveur de Wikidata
    
    headers = {"User-Agent": "WikidataImageDownloader/1.0 (contact: yanis.pires2004@gmail.com)"} # Permet d'éviter d'être bloqué par le serveur de Wikidata en raison d'un taux de requêtes élevé
    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)
    
    if request.status_code == 200:
        os.makedirs("./images", exist_ok=True)
        filepath = os.path.join("./images", os.path.basename(url))
        
        with open(filepath, "wb") as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
            
      # Pause de 1 seconde entre les téléchargements
    return request.status_code


dataframe["filename"] = dataframe.image.apply(os.path.basename)
dataframe["filename"].value_counts()


dataframe.image.apply(download_image)

0    200
1    200
2    200
3    200
4    200
5    200
6    200
7    200
8    200
9    200
Name: image, dtype: int64